In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
%matplotlib inline
os.getcwd()

In [1]:
train_c=pd.read_csv('/kaggle/input/ihsmarkit-hackathon-june2020/train_data.csv')
test_c=pd.read_csv('/kaggle/input/ihsmarkit-hackathon-june2020/oos_data.csv')
#sample=pd.read_csv('/kaggle/input/ihsmarkit-hackathon-june2020/oos_data.csv')
#train_c=pd.read_csv('/Users/dharmendraverma/Downloads/Python/ihsmarkit-hackathon-june2020/train_data.csv',encoding='utf-8')
#test_c=pd.read_csv('/Users/dharmendraverma/Downloads/Python/ihsmarkit-hackathon-june2020/oos_data.csv')

train=train_c.copy()
test=test_c.copy()

train_c=pd.read_csv('/Users/dharmendraverma/Downloads/Python/ihsmarkit-hackathon-june2020/train_data.csv')
test=pd.read_csv('/Users/dharmendraverma/Downloads/Python/ihsmarkit-hackathon-june2020/oos_data.csv')

train.head()
train=train.copy()
test.info()

In [1]:
train.info()

In [1]:
[col for col in train.columns if train[col].dtype=='object']


In [1]:
#columns below that contain some unknown values
TRAIN=train.replace('unknown',np.nan)
#.groupby('Body_Type')['Body_Type'].count().sum()
#train=train[train['Fuel_Type']!='unknown']
#train=train[train['Brand']!='unknown']
#train=train[train['Body_Type']!='unknown']
train.isnull().sum()

In [1]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):

        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 

        in column.

        Columns of other types are imputed with mean of column.

        """

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]

            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],

            index=X.columns)

        return self

    def transform(self, X, y=None):

        return X.fillna(self.fill)

In [1]:
train.groupby('Body_Type')['Body_Type'].count()
train[train['Body_Type']=='unknown']["Body_Type"]
#train.groupby('Body_Type')['Body_Type'].count()


In [1]:
train['power_to_wt']=train['Engine_KW']/train['Curb_Weight']
test['power_to_wt']=test['Engine_KW']/test['Curb_Weight']
train['volume']=train['Length']*train['Width']*train['Height']
test['volume']=test['Length']*test['Width']*test['Height']
train['density']=train['Curb_Weight']/train['volume']
test['density']=test['Curb_Weight']/test['volume']

In [1]:
train=train.loc[train['Price_USD']<500000]

train=train.loc[train['Price_USD']>5000]

In [1]:
train.groupby(train.Nameplate.str.contains('unknown'))['Nameplate'].count()

In [1]:
train.groupby(train.Driven_Wheels.str.contains('unknown'))['Driven_Wheels'].count()

In [1]:
train.groupby(train.Transmission.str.contains('unknown'))['Transmission'].count()

In [1]:
train.groupby(train.Turbo.str.contains('unknown'))['Turbo'].count()

In [1]:
train.groupby(train.Fuel_Type.str.contains('unknown'))['Fuel_Type'].count()

In [1]:
train.groupby(train.PropSysDesign.str.contains('unknown'))['PropSysDesign'].count()

In [1]:
train.groupby(train.Plugin.str.contains('unknown'))['Plugin'].count()

In [1]:
train.isnull().sum()

In [1]:
train.corr()

In [1]:
train.columns

In [1]:
train['vehicle_life']=train['year']-train['Generation_Year']
test['vehicle_life']=test['year']-test['Generation_Year']
train['vehicle_life'].loc[train['vehicle_life']<0]=0
train.vehicle_life.describe()

In [1]:
train['date']=pd.to_datetime(train['date']).dt.date
train['date'].head()

In [1]:
plt.plot(train[train['country_name']=='China'].groupby(['year'])['vehicle_id'].count())
plt.xlabel('Year')
plt.ylabel('Count of Vehicles')
plt.title('Vehicle Sales')
plt.plot(train[train['country_name']=='Germany'].groupby(['year'])['vehicle_id'].count())
plt.plot(train[train['country_name']=='USA'].groupby(['year'])['vehicle_id'].count())


In [1]:
train_cat=[col for col in train.columns if train[col].dtype=='object']

train_num=[col for col in train.columns if train[col].dtype in ['int64','float64']]# or train[col].dtype in 'float64']]

In [1]:
train[train_cat].nunique()
train_num

In [1]:
train.groupby(['Brand'])['vehicle_id'].count().sort_values(ascending=False).head(20)

In [1]:
cat_data=train[train_cat].drop(['date','Brand','Nameplate'],axis=1)

from sklearn.preprocessing import StandardScaler

train_X=train[train_num].drop(['Price_USD','vehicle_id','Generation_Year','year'], axis=1)
train_y=train['Price_USD']
test_X=test[['Length','Height','Width','Engine_KW','No_of_Gears','Curb_Weight','CO2','Fuel_cons_combined','vehicle_life','power_to_wt', 'volume', 'density']]

scaler=StandardScaler()
train_X_scaled=scaler.fit_transform(train_X)
test_X_scaled=scaler.fit_transform(test_X)
train_X_scaled

In [1]:
import numpy as np
def mape(y_true, y_pred):
    y_val = np.maximum(np.array(y_true), 1e-8)
    return (np.abs(y_true -y_pred)*100/y_val).mean()

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

linear=LinearRegression()
linear.fit(train_X_scaled, train_y)
y_predict=linear.predict(test_X_scaled)

print('Mean absolute percent error for Linear Regression')


from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest=train_test_split(train_X_scaled, train_y, test_size=.3, random_state=0)
linear=LinearRegression()
linear.fit(Xtrain, ytrain)
ypredict=linear.predict(Xtest)
print('Mean absolute percent error for Linear Regression',mape(ytest, ypredict))


tree=DecisionTreeRegressor()
tree.fit(Xtrain, ytrain)
ypredict_tree=tree.predict(Xtest)
print('Mean absolute percent error for Decision Tree Regression',mape(ytest, ypredict_tree))


n_estimators=[10,10,100,1000]
max_depth=[2,4,6,8,10]
for n in n_estimators:
    for m in max_depth:
        random=RandomForestRegressor(n_estimators=n, max_depth=m)
        random.fit(Xtrain, ytrain)
        ypredict_random=random.predict(Xtest)
        print('Mean absolute percent error for Linear Regression for n:{0},m:{1}'.format(n,m) ,mape(ytest, ypredict_random))


random=RandomForestRegressor(random_state=0)
random.fit(Xtrain, ytrain)
ypredict_random=random.predict(Xtest)
print('Mean absolute percent error for RandomForestRegressor',mape(ytest, ypredict_random))
random

ypredict_OOS=random.predict(test_X_scaled)
ypredict_OOS.shape

from sklearn.compose import TransformedTargetRegressor
tt = TransformedTargetRegressor(regressor=RandomForestRegressor(),func=np.log, inverse_func=np.exp)
tt.fit(Xtrain, ytrain)
y_pred_tt=tt.predict(Xtest)
y_pred_tr=tt.predict(Xtrain)
plt.scatter(ytrain, y_pred_tr, c='b')
plt.scatter(ytest, y_pred_tt, c='r')
mape(ytest, y_pred_tt)

ypredict_OO=pd.DataFrame(ypredict_OOS,columns=['Price_USD'])

l=[]
for i in test['vehicle_id']:
    l.append(i)
ypredict_OO.index=l
ypredict_OO.index.name='vehicle_id'


output=ypredict_OO.head()

In [1]:
#output=ypredict_OO.to_csv('submission.csv')

In [1]:
train[train_cat].columns

In [1]:
print('train["Body_Type"] - ',train['Body_Type'].nunique())#.sort()
print('test["Body_Type"] - ',test['Body_Type'].nunique())

for col in train[train_cat].columns:
    print("{0} unique values in {1}, {2} are".format(col,'Train set', 'Test set'),train[col].nunique(),", ", test[col].nunique()
         , "and are equal", train[col].nunique()== test[col].nunique() ,"\n")

In [1]:
#checking for categorical variables where number of categories in each variable are same in train and test set
cat_to_be_used=[col for col in train[train_cat] if train[col].nunique()== test[col].nunique()]


In [1]:
#using label encoding
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [1]:
le=LabelEncoder()
onehot=OneHotEncoder()
print(train_cat )
cat_to_be_used

In [1]:
for col in cat_to_be_used:
    # Using whole data to form an exhaustive list of levels
    data=train[col].append(test[col])
    le.fit(data.values)
    train[col]=le.transform(train[col])
    test[col]=le.transform(test[col])

In [1]:
print(cat_to_be_used)

In [1]:
from sklearn.preprocessing import StandardScaler

col_scaled=train_num+cat_to_be_used
remove_items=['vehicle_id', 'Generation_Year','year','Price_USD','No_of_Gears']
#col_scaled=col_scaled.remove('vehicle_id')
col_keep=[x for x in col_scaled if x not in remove_items]
training_labeled_X=train[col_keep]
y=train['Price_USD']
test_labeled_X=test[col_keep]
col_keep

In [1]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler=MinMaxScaler()
train_labeled_X_scaled=scaler.fit_transform(training_labeled_X)
test_labeled_X_scaled=scaler.fit_transform(test_labeled_X)
test_labeled_X_scaled

In [1]:
#spliting training data in train and test for model creation
from sklearn.model_selection import train_test_split
trX, tsX, trY,tsY=train_test_split(train_labeled_X_scaled, y, test_size=.3, random_state=0)
from sklearn.ensemble import RandomForestRegressor
random=RandomForestRegressor(random_state=0) 
random.fit(trX, trY)
pred_Y=random.predict(tsX)
pred_y_r=random.predict(trX)
plt.scatter(trY, pred_y_r, c='b')
plt.scatter(tsY,pred_Y, c='r')
mape(tsY, pred_Y)

In [1]:
from sklearn.compose import TransformedTargetRegressor
tt = TransformedTargetRegressor(regressor=RandomForestRegressor(),func=np.log, inverse_func=np.exp)
tt.fit(trX, trY)
y_pred_tt1=tt.predict(tsX)
y_pred_tr1=tt.predict(trX)
plt.scatter(trY, y_pred_tr1, c='b')
plt.scatter(tsY, y_pred_tt1, c='r')
mape(tsY, y_pred_tt1)

In [1]:
predY=tt.predict(test_labeled_X_scaled)

predY=pd.DataFrame(predY,columns=['Price_USD'])
l=[]
for i in test['vehicle_id']:
    l.append(i)
predY.index=l
predY.index.name='vehicle_id'
predY.head()

In [1]:
from xgboost import XGBRegressor, plot_importance  
xgb_model1 = XGBRegressor()
xgb_model1.fit(trX, trY, verbose=False)
pred_y_xgb1=xgb_model1.predict(tsX)
mape(tsY,pred_y_xgb1)

In [1]:
xgb_model2 = XGBRegressor(n_estimators=1000)
xgb_model2.fit(trX, trY, early_stopping_rounds=5, 
             eval_set=[(tsX, tsY)], verbose=False)
pred_y_xgb2=xgb_model2.predict(tsX)
mape(tsY,pred_y_xgb2)

In [1]:

xgb_model3 = XGBRegressor(n_estimators=1000, learning_rate=0.1)
xgb_model3.fit(trX, trY, early_stopping_rounds=5, 
             eval_set=[(tsX, tsY)], verbose=False)
pred_y_xgb3=xgb_model3.predict(tsX)
mape(tsY,pred_y_xgb3)

In [1]:
predY=tt.predict(test_labeled_X_scaled)

predY=pd.DataFrame(predY,columns=['Price_USD'])
l=[]
for i in test['vehicle_id']:
    l.append(i)
predY.index=l
predY.index.name='vehicle_id'
predY.head()

In [1]:
output=predY.to_csv('submission.csv',float_format='%g')